In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup

your_jobplanet_email = ''
your_jobplanet_password = ''
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
url = 'https://www.jobplanet.co.kr/welcome/index'
driver.get(url)

# 로그인창 접속하기
driver.find_element(By.CSS_SELECTOR, 'a.btn_txt.login').click()

# 이메일 입력하기
search_window = driver.find_element(By.CSS_SELECTOR, 'input#user_email')
search_window.send_keys(your_jobplanet_email)

# 비밀번호 입력하기
search_window = driver.find_element(By.CSS_SELECTOR, 'input#user_password')
search_window.send_keys(your_jobplanet_password)

# 로그인
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, 'button.btn_sign_up').click()


# 기업 랭킹 접속
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, 'li.companies').click()

# IT/웹/통신
time.sleep(3)
driver.find_element(By.LINK_TEXT, 'IT/웹/통신').click()





In [13]:
# 회사리스트 url에 바로 접속하기 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup

your_jobplanet_email = ''
your_jobplanet_password = ''
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
url = 'https://www.jobplanet.co.kr/welcome/index'
driver.get(url)

# 로그인창 접속하기
driver.find_element(By.CSS_SELECTOR, 'a.btn_txt.login').click()

# 이메일 입력하기
search_window = driver.find_element(By.CSS_SELECTOR, 'input#user_email')
search_window.send_keys(your_jobplanet_email)

# 비밀번호 입력하기
search_window = driver.find_element(By.CSS_SELECTOR, 'input#user_password')
search_window.send_keys(your_jobplanet_password)

# 로그인
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, 'button.btn_sign_up').click()




In [14]:
dic_company = {'company_id':[], 'name':[],'jp_url' : [],'category': [], 'description':[],'link':[],'salary':[],'logo':[],'location':[],'CEO':[]}

In [15]:

# 회사리스트 페이지에서 들고올 정보 : company_id, name, jp_url, logo, salary 

company_id = 91
total_links = 31
for k in range(10, total_links):

    # 페이지 클릭 ( 2, 3, 4, 5 ..)
    driver.get('https://www.jobplanet.co.kr/companies/by_industry/700?page='+str(k))
    time.sleep(4)

    for i in range(10):
        companies = driver.find_elements(By.CSS_SELECTOR, 'dt.us_titb_l3')
        dic_company['name'].append(companies[i].text.split(' ')[0]) # name 파싱
        dic_company['company_id'].append(company_id)
        company_id += 1
    

    # jp_url 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    for i in soup.select('section.company.content_ty3'):
        dic_company['jp_url'].append('https://www.jobplanet.co.kr'+ i.select('a')[1].attrs['href'])

    # logo 파싱
    for img in driver.find_elements(By.CSS_SELECTOR, 'div.section_group img'):
        dic_company['logo'].append(img.get_attribute('src'))

    # salary 파싱
    for i in driver.find_elements(By.CSS_SELECTOR, 'strong.notranslate'):
        dic_company['salary'].append(i.text)

    
    

In [16]:
# 처음 뜨는 팝업창 제거를 위한 티몬 접속
time.sleep(3)
driver.get('https://www.jobplanet.co.kr/companies/50695/info/%ED%8B%B0%EB%AA%AC?')
time.sleep(5)
driver.find_element(By.CSS_SELECTOR, 'button.btn_close_x_ty1').click()
time.sleep(3)


for url in dic_company['jp_url']:
    
    driver.get(url)

    # 뉴스룸 (기업정보) 접속
    time.sleep(10)
    # driver.find_element(By.LINK_TEXT, '뉴스룸').click()
    driver.find_element(By.XPATH,'//*[@id="viewCompaniesMenu"]/ul/li[1]/a').send_keys(Keys.ENTER)
    time.sleep(10)

    try :
        displayOk = driver.find_element(By.CSS_SELECTOR,'div.basic_info_sec button').is_displayed()
        print(displayOk, type(displayOk))
        if displayOk :
            driver.find_element(By.CSS_SELECTOR,'div.basic_info_sec button').send_keys(Keys.ENTER)

    except:
        pass

    #회사 산업군 파싱
    info = driver.find_element(By.CSS_SELECTOR, 'div.info').text
    info = info.split(' · ')
    print(info[0])
    dic_company['category'].append(info[0])

    # 회사 대표 파싱
    CEO = driver.find_elements(By.CSS_SELECTOR, 'dd')[0].text
    dic_company['CEO'].append(CEO)
    print(CEO)
    # 회사 주소 파싱
    location = driver.find_elements(By.CSS_SELECTOR, 'dd')[2].text
    dic_company['location'].append(location)
    print(location)
    # 회사 웹사이트 파싱
    link = driver.find_elements(By.CSS_SELECTOR, 'dd')[3].text
    dic_company['link'].append(link)
    print(link)
    #회사 소개 파싱
    description = driver.find_elements(By.CSS_SELECTOR, 'dd')[5].text
    dic_company['description'].append(description)
    print(description)
    

    

dic_company
df_company = pd.DataFrame(dic_company)
column_names = ['company_id', 'name','jp_url' ,'category', 'description','link','salary','logo','location','CEO']
df_company.to_csv(r".\company.csv", index= False,columns=column_names,na_rep='NULL', encoding='utf-8',mode='a')

IT/웹/통신
강석립
서울 송파구 잠실동, 올림픽로35길 123 10층 (신천동, 향군타워)
http://www.miracom-inc.com
미라콤은 선도적 기술역량을 바탕으로 라인업 된 MES, Middleware, Healthcare 등의 특화 솔루션을 비롯하여 기업 정보시스템 개발 및 운영에 이르는 토탈 ICT 서비스를 제공합니다. 창사 이래 20년간 고부가가치 솔루션과 서비스 개발에 집중해 온 미라콤은 풍부한 ICT 지식과 경험을 갖춘 2,000여명의 전문가가 글로벌 선도기술과 창의적 Insight로 고객가치 창출을 실현하고 있습니다.
True <class 'bool'>
IT/웹/통신
서정식
서울 강남구 테헤란로 510
www.hyundai-autoever.com
현대오토에버는 현대자동차 그룹의 IT 전문 계열사로서 2000년 4월 설립하여, 그룹의 주력사업인 자동차, 부품, 철강, 물류, 중공업, 건설, 금융 분야 등에서 축적된 IT서비스 노하우와 선진 기술력을 바탕으로 지속적으로 성장해왔습니다.
현대오토에버는 ‘고객 가치 증대를 위한 IT서비스 기업’이라는 이념 아래 고객기업의 정보 선진화를 제 일의 핵심가치로 여기며, 최고의 정보 기술과 전문가를 육성하고 있습니다. 그리고, 전문 IT서비스 뿐만 아니라 제조 IT 및 엔지니어링 서비스, 임베디드 S/W개발, 이러닝 등의 분야에서 최고 수준의 기술력을 바탕으로 차별화된 서비스를 제공하고 있습니다.
또한, 그룹내 IT 질적 향상을 위해 정보기술 표준화 및 통합화를 구현하여 업무의 효율성을 향상시키고, 품질 강화를 통하여 경영전략의 성공적 수행을 가능케 하며, 아울러 이를 통한 고객사의 미래 가치 발굴을 지원하는 최고의 파트너입니다.
IT/웹/통신
정우진
경기 성남시 분당구 대왕판교로645번길 16
www.nhn.com
NHN
NHN은 오늘을 만드는 기술, 세상과 호흡하는 IT 기술로 우리의 일상을 풍요롭게 만듭니다. 게임, 기술, 결제, 커머스, 콘텐츠 5대 핵심 사업을 통해 2030년까지 글로벌 테크

# 마지막 : 로고 img 저장하기

In [ ]:
## path_folder의 경로는 각자 저장할 폴더의 경로를 적어줄 것(ex.img_download)
path_folder = r".\img_download"

# 이미지 다운로드
i = 1

for link in dic_company['logo']:  
    r = requests.get(link)
    print(r)
    with open(path_folder+ f'/{i}.jpg',"wb") as outfile:
        outfile.write(r.content)
    i+=1    
        

# 크롤링 함수 만들기

In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup

def crawling_company(company_id, start_page, end_page):
    dic_company = {'company_id':[], 'name':[],'jp_url' : [],'category': [], 'description':[],'link':[],'salary':[],'logo':[],'location':[],'CEO':[]}
    your_jobplanet_email = ''
    your_jobplanet_password = ''
    s=Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=s)
    url = 'https://www.jobplanet.co.kr/welcome/index'
    driver.get(url)

    # 로그인창 접속하기
    driver.find_element(By.CSS_SELECTOR, 'a.btn_txt.login').click()

    # 이메일 입력하기
    search_window = driver.find_element(By.CSS_SELECTOR, 'input#user_email')
    search_window.send_keys(your_jobplanet_email)

    # 비밀번호 입력하기
    search_window = driver.find_element(By.CSS_SELECTOR, 'input#user_password')
    search_window.send_keys(your_jobplanet_password)

    # 로그인
    time.sleep(3)
    driver.find_element(By.CSS_SELECTOR, 'button.btn_sign_up').click()
    
    # 회사리스트 페이지에서 들고올 정보 : company_id, name, jp_url, logo, salary 
    time.sleep(5)
    
    for k in range(start_page, end_page + 1):

        # 페이지 클릭 ( 2, 3, 4, 5 ..)
        driver.get('https://www.jobplanet.co.kr/companies/by_industry/700?page='+str(k))
        time.sleep(10)

        for i in range(10):
            companies = driver.find_elements(By.CSS_SELECTOR, 'dt.us_titb_l3')
            dic_company['name'].append(companies[i].text.split(' ')[0]) # name 파싱
            dic_company['company_id'].append(company_id)
            company_id += 1
        

        # jp_url 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        for i in soup.select('section.company.content_ty3'):
            dic_company['jp_url'].append('https://www.jobplanet.co.kr'+ i.select('a')[1].attrs['href'])

        # logo 파싱
        for img in driver.find_elements(By.CSS_SELECTOR, 'div.section_group img'):
            dic_company['logo'].append(img.get_attribute('src'))

        # salary 파싱
        for i in driver.find_elements(By.CSS_SELECTOR, 'strong.notranslate'):
            dic_company['salary'].append(i.text)

    # 처음 뜨는 팝업창 제거를 위한 티몬 접속
    time.sleep(3)
    driver.get('https://www.jobplanet.co.kr/companies/50695/info/%ED%8B%B0%EB%AA%AC?')
    time.sleep(5)
    driver.find_element(By.CSS_SELECTOR, 'button.btn_close_x_ty1').click()
    time.sleep(3)

    # 각 회사의 잡플래닛 페이지 접속
    for url in dic_company['jp_url']:
        
        driver.get(url)

        # 뉴스룸 (기업정보) 접속
        time.sleep(10)
        # driver.find_element(By.LINK_TEXT, '뉴스룸').click()
        driver.find_element(By.XPATH,'//*[@id="viewCompaniesMenu"]/ul/li[1]/a').send_keys(Keys.ENTER)
        time.sleep(10)

        try :
            displayOk = driver.find_element(By.CSS_SELECTOR,'div.basic_info_sec button').is_displayed()
            print(displayOk, type(displayOk))
            if displayOk :
                driver.find_element(By.CSS_SELECTOR,'div.basic_info_sec button').send_keys(Keys.ENTER)

        except:
            pass

        #회사 산업군 파싱
        info = driver.find_element(By.CSS_SELECTOR, 'div.info').text
        info = info.split(' · ')
        
        dic_company['category'].append(info[0])

        # 회사 대표 파싱
        CEO = driver.find_elements(By.CSS_SELECTOR, 'dd')[0].text
        dic_company['CEO'].append(CEO)
        
        # 회사 주소 파싱
        location = driver.find_elements(By.CSS_SELECTOR, 'dd')[2].text
        dic_company['location'].append(location)
        
        # 회사 웹사이트 파싱
        link = driver.find_elements(By.CSS_SELECTOR, 'dd')[3].text
        dic_company['link'].append(link)
        
        #회사 소개 파싱
        description = driver.find_elements(By.CSS_SELECTOR, 'dd')[5].text
        dic_company['description'].append(description)
        

    df_company = pd.DataFrame(dic_company)
    column_names = ['company_id', 'name','jp_url' ,'category', 'description','link','salary','logo','location','CEO']
    df_company.to_csv(r".\company.csv", index= False,header=False,columns=column_names,na_rep='NULL', encoding='utf-8',mode='a')
    
    return df_company

In [ ]:
crawling_company(91, 10, 11)

True <class 'bool'>
True <class 'bool'>


,company_id,name,jp_url,category,description,link,salary,logo,location,CEO
0,91,(주)위메이드,https://www.jobplanet.co.kr/companies/60634/in...,IT/웹/통신,-,www.wemade.com,"4,884",https://jpassets.jobplanet.co.kr/production/up...,"경기 성남시 분당구 대왕판교로644번길 49, 코리아벤처타운업무시설 비블럭",장현국
1,92,한국지능정보사회진흥원,https://www.jobplanet.co.kr/companies/89015/in...,IT/웹/통신,□ 미션 : 우리는 ICT로 사회현안을 해결하고 국가미래를 열어간다\n□ 비전 : ...,www.nia.or.kr,"4,312",https://jpassets.jobplanet.co.kr/production/up...,대구광역시 동구 첨단로 53,문용식
2,93,나비스오토모티브시스템즈(주),https://www.jobplanet.co.kr/companies/42796/in...,IT/웹/통신,완성차용 OEM내비게이션SW개발\n한국 전자지도 개발,www.navis-ams.com,"4,758",https://jpassets.jobplanet.co.kr/production/up...,부산 해운대구 우동 센텀중앙로 78 센텀그린타워 8층,정희석
3,94,한국오라클(유),https://www.jobplanet.co.kr/companies/3277/inf...,IT/웹/통신,-,www.oracle.co.kr,"6,537",https://jpassets.jobplanet.co.kr/production/up...,"서울 강남구 영동대로 517, 12층, 15층, 16층, 36층 (삼성동,아셈타워)",문건
4,95,줌인터넷(주),https://www.jobplanet.co.kr/companies/55808/in...,IT/웹/통신,"[줌인터넷을 소개합니다]\n\n""더욱 쉽고 편리하게 금융과 콘텐츠 경험의 판을 바꾸...",http://www.zuminternet.com/,"4,531",https://jpassets.jobplanet.co.kr/production/up...,서울 서초구 서초동 반포대로 3 7층 (서초동 1464-30 이스트빌딩 7층),이성현
5,96,(주)웹젠,https://www.jobplanet.co.kr/companies/60745/in...,IT/웹/통신,"(주)웹젠은 2000년 설립 이후, 국내 최초 Full 3D 온라인 게임 '뮤 온라...",http://company.webzen.com,"3,997",https://jpassets.jobplanet.co.kr/production/up...,"경기도 성남시 분당구 판교로 242 (삼평동, 판교디지털센터 B동)",김태영
6,97,이씨이십일(주),https://www.jobplanet.co.kr/companies/58813/in...,IT/웹/통신,"""EC21 makes the world trade easier!""\n\nEC21은 ...",www.ec21.co.kr,"4,219",https://jpassets.jobplanet.co.kr/production/up...,"서울특별시 강남구 삼성동 무역센터 트레이드타워 5F, 8F, 11F",송영록
7,98,어니컴(주),https://www.jobplanet.co.kr/companies/21282/in...,IT/웹/통신,1. OVERVIEW\n\n어니컴 주식회사는 설립이래 항상 시대의 주역이 되기 위해...,www.onycom.com,"3,338",https://jpassets.jobplanet.co.kr/production/up...,"서울 용산구 두텁바위로 21, 5층","이석호, 양인집"
8,99,에스케이브로드밴드(주),https://www.jobplanet.co.kr/companies/20626/in...,IT/웹/통신,에스케이브로드밴드(주)는 케이블 TV서비스 및 초고속 인터넷 서비스 사업 부문을 영...,www.skbroadband.com,"5,745",https://jpassets.jobplanet.co.kr/production/up...,서울 중구 퇴계로 24 남산그린빌딩,최진환
9,100,(주)제이슨그룹,https://www.jobplanet.co.kr/companies/91388/in...,IT/웹/통신,-,https://jasonproup.co.kr,"3,743",https://jpassets.jobplanet.co.kr/production/up...,"서울 용산구 서빙고로 17 센트럴파크타워 26층,27층",정진영
